<a href="https://colab.research.google.com/github/tomunizua/summative_bananaguard/blob/main/Summative_Intro_to_ml_TheodoraOmunizua_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimization Techniques in Machine Learning

Objective: This assignment aims to explore implementation or Machine Learning Models with regularization, optimization and Error analysis  techniques used in machine learning to improve models' performance, convergence speed, and efficiency..

A Notebook detailing the following

* Project name
* Clear out puts from cells






**Instructions**

1. Acquire a dataset suitable for ML tasks as per your proposal.
2. Implement a simple machine learning model based on neural networks on the chosen dataset without any defined optimization techniques. (Check instructions)
3. Implement and compare the model's performance after applying 3 to 4 disntict combinations regularization and optimization techniques.
4. Discuss the results on the README file.
5. Make predictions using test data
7. Implement error analysis techniques and ensure there is: F1-Score, Recall, Precision, RUC a confusion matrix using plotting libraries (not verbose)

Submit notebook to github repo




# Case Study and Implementation




In [7]:
#Import Necessary Libraries

import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
from PIL import Image
import tensorflow as tf

# For PyTorch and YOLOv5
import torch
from torchvision import datasets, models, transforms
#!pip install yolov5
from yolov5 import YOLOv5


# The Dataset
> ***Brief Description:***
State the Problem and A short Description of the data


In [2]:
#!pip install roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="D84UstygUMTvzYCf2Rhd")
project = rf.workspace("projectia-khxte").project("segment-tyeq6")
version = project.version(1)
dataset = version.download("yolov5")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Segment-1 in yolov5pytorch:: 100%|██████████| 2812/2812 [00:00<00:00, 3361.49it/s]


In [3]:
#TO DO: Load Data (Seprate into: Train, Validation and test sets)

train_image_dir = '/content/Segment-1/train/images'
train_label_dir = '/content/Segment-1/train/labels'

valid_image_dir = '/content/Segment-1/valid/images'
valid_label_dir = '/content/Segment-1/valid/labels'

test_image_dir = '/content/Segment-1/test/images'
test_label_dir = '/content/Segment-1/test/labels'

def get_file_paths(image_dir, label_dir):
    image_paths = [os.path.join(image_dir, fname) for fname in os.listdir(image_dir) if fname.endswith('.jpg')]
    label_paths = [os.path.join(label_dir, fname) for fname in os.listdir(label_dir) if fname.endswith('.txt')]
    return image_paths, label_paths

train_images, train_labels = get_file_paths(train_image_dir, train_label_dir)
valid_images, valid_labels = get_file_paths(valid_image_dir, valid_label_dir)
test_images, test_labels = get_file_paths(test_image_dir, test_label_dir)

def preprocess_labels(label_path):
    with open(label_path, 'r') as file:
        labels_str = file.read().split()
    # Handle potential empty label files:
    if not labels_str:
        return np.array([], dtype=np.float32)  # Or a suitable default value
    try:
      labels = np.array([float(x) for x in labels_str], dtype=np.float32)
    except ValueError as e:
      print(f"Error converting labels in {label_path}: {e}")
      print(f"Problematic string: {labels_str}")
      return np.array([], dtype=np.float32) #Or a suitable default value
    return labels


train_labels = [preprocess_labels(path) for path in train_labels]
valid_labels = [preprocess_labels(path) for path in valid_labels]
test_labels = [preprocess_labels(path) for path in test_labels]

def load_image(image_path):
    image_string = tf.io.read_file(image_path)
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    image_resized = tf.image.resize(image_decoded, [128, 128])
    return image_resized

def create_dataset(images, labels):
    ragged_labels = tf.ragged.constant(labels)

    dataset = tf.data.Dataset.from_tensor_slices({"image": images, "label": ragged_labels})

    def process_path(data):
        image_path = data["image"]
        label = data["label"]
        image = load_image(image_path)
        return {"image": image, "label": label}

    dataset = dataset.map(process_path)

    dataset = dataset.padded_batch(
        batch_size=32,
        padded_shapes={"image": [128, 128, 3], "label": [None]},
        padding_values={"image": tf.constant(0.0, dtype=tf.float32), "label": tf.constant(-1.0, dtype=tf.float32)}
    )

    dataset = dataset.shuffle(buffer_size=100)
    return dataset


train_dataset = create_dataset(train_images, train_labels)
valid_dataset = create_dataset(valid_images, valid_labels)
test_dataset = create_dataset(test_images, test_labels)


#SECTION 1: Model Architecture:



```
TODO: Insert an image with the Model architecture here.Replace the image Below
```
> <img src="https://miro.medium.com/v2/resize:fit:640/format:webp/1*v1ohAG82xmU6WGsG2hoE8g.png" alt="?" style="width:25px"/>




#Task: Define a function that creates models without and With specified Optimization techniques


In [17]:
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

# Define a simple neural network model
def define_noneoptimizer_model():
  model = Sequential()
  model.add(Flatten(input_shape=(128, 128, 3)))  # Adjust input shape accordingly
  model.add(Dense(64, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

# Compile the model without specifying any optimizer
  model.compile(optimizer=None, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model on the training dataset
  history = model.fit(train_dataset, validation_data=valid_dataset, epochs=10)

# Define and train the simple model
  noneoptimizer_model = define_noneoptimizer_model()

# Evaluate the model on the test dataset
  results = noneoptimizer_model.evaluate(test_dataset)
  print(f"Test Accuracy: {results[1]*100:.2f}%")

# def define_model(optimization: string, regularization_datatype, early_stopping: bool, dropout: float, learning_rate: float):
#   model= None
#   model.add(None)
#   #TO DO: Add more layers as per architecture
#   model.add(None) # Last Layer
#   model.compile(optimizer = optimizerNone)
#   model.fit(None)
#   return model

# Task: Print out the Final Model Accuracy and plot the Loss curve

In [20]:
def loss_curve_plot(history):
  epochs = range(1, len(history.history['loss']) + 1)
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  plt.plot(epochs, loss, 'bo', label='Training loss')
  plt.plot(epochs, val_loss, 'r', label='Validation loss')
  plt.title('Training and Validation Loss')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()
  plt.show()

# SECTION 2: Optimization and Regularization Combinations
At this point you should now create models that combine various optimization techniques
As done before make sure to plot out the loss curve and the accuracy and loss in verbose

In [ ]:
#TODO:
model_2 = define_model('Adam', None)
loss_curve_plot(model_2):
#print out confusion matrix and error analysis metrics after the cell

In [ ]:
#TODO:
model_3 = define_model('RMSPop',None)
loss_curve_plot(model_3):
#print out confusion matrix and error analysis metrics after the cell

In [ ]:
#TODO:
model_4 = define_model(None)
loss_curve_plot(model_4):
#print out confusion matrix and error analysis metrics after the cell

#Task: Make Predictions using the best saved model


Create a confusion Matrix and F1 score for both Models. Ensure outputs for the cells are visible

Finally, Make predictions using the best model. By the time you get to this cell you may realise at some point you needed to save the model so that you cal load it later

In [ ]:
def make_predictions(model_path, X):

    # Load the model
    model = load_model(None)
    # Make predictions
    predictions = None
    # Convert probabilities to binary labels (0 or 1)

    return predictions

#Modify the code appropriately

In [ ]:
model_path = None
make_predictions(None)

Congratulations!!
